In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Define the data loading function (adapted from your Streamlit app)
def load_data():
    """Loads all e-commerce datasets from the specified file path."""
    # Define file paths
    path = "E-commerce/"
    
    # Load datasets
    # The corrected line
    orders = pd.read_csv(
        path + "orders_dataset.csv", 
        parse_dates=[
            "order_purchase_timestamp", 
            "order_approved_at",          # <-- Add this
            "order_delivered_carrier_date", # <-- Add this
            "order_delivered_customer_date",
            "order_estimated_delivery_date"
        ]
    )
    order_items = pd.read_csv(path + "order_items_dataset.csv")
    products = pd.read_csv(path + "products_dataset.csv")
    cat_trans = pd.read_csv(path + "product_category_name_translation.csv")
    customers = pd.read_csv(path + "customers_dataset.csv")
    payments = pd.read_csv(path + "order_payments_dataset.csv")
    reviews = pd.read_csv(path + "order_reviews_dataset.csv", parse_dates=["review_creation_date", "review_answer_timestamp"])
    mql = pd.read_csv(path + "marketing_qualified_leads_dataset.csv", parse_dates=["first_contact_date"])
    deals = pd.read_csv(path + "closed_deals_dataset.csv", parse_dates=["won_date"])
    geolocation = pd.read_csv(path + "geolocation_dataset.csv")
    sellers = pd.read_csv(path + "sellers_dataset.csv")
    
    return orders, order_items, products, cat_trans, customers, payments, reviews, mql, deals, geolocation, sellers

# --- Load all data into variables ---
orders, order_items, products, cat_trans, customers, payments, reviews, mql, deals, geolocation, sellers = load_data()

# --- Perform initial merge to create the 'items' DataFrame ---
# This is the same merge helper from your app
items = order_items.merge(products[["product_id", "product_category_name"]], on="product_id", how="left")
items = items.merge(cat_trans, on="product_category_name", how="left")

# --- Verification Step ---
# You can now use all these DataFrames for exploration.
# For example, let's display the first few rows of the 'items' DataFrame to confirm it's loaded.
print("Data loaded and initial merge complete. The 'items' DataFrame is ready:")
items.head()

Data loaded and initial merge complete. The 'items' DataFrame is ready:


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_category_name_english
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cool_stuff,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,pet_shop,pet_shop
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,moveis_decoracao,furniture_decor
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,perfumaria,perfumery
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,ferramentas_jardim,garden_tools


In [6]:
# Code to run in your notebook
print(orders['order_status'].unique())

['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'created' 'approved']


In [7]:
# --- 1. Filter for relevant orders and calculate stage durations ---
# Ensure all timestamps needed are present
completed_orders = orders[
    orders['order_status'] == 'delivered'
].dropna(subset=[
    'order_purchase_timestamp', 
    'order_approved_at', 
    'order_delivered_carrier_date', 
    'order_delivered_customer_date'
])

# Calculate durations in days
# Using .dt.total_seconds() / (24 * 3600) is more precise for timedelta
completed_orders['processing_time'] = (completed_orders['order_approved_at'] - completed_orders['order_purchase_timestamp']).dt.total_seconds() / (24 * 3600)
completed_orders['seller_handling_time'] = (completed_orders['order_delivered_carrier_date'] - completed_orders['order_approved_at']).dt.total_seconds() / (24 * 3600)
completed_orders['carrier_shipping_time'] = (completed_orders['order_delivered_customer_date'] - completed_orders['order_delivered_carrier_date']).dt.total_seconds() / (24 * 3600)

# --- 2. Merge with product category information ---
# We use the 'items' helper DataFrame you already created
order_stage_times = completed_orders.merge(items[['order_id', 'product_category_name_english']], on='order_id', how='left')

# Drop orders that might have multiple different item categories to keep the analysis simple
order_stage_times = order_stage_times.drop_duplicates(subset=['order_id'])


# --- 3. Aggregate to find average times per category ---
# Focusing on the top 10 categories by revenue for clarity
top_10_cats = items.groupby('product_category_name_english')['price'].sum().nlargest(10).index

# Filter for top 10 categories
avg_times_by_cat = order_stage_times[
    order_stage_times['product_category_name_english'].isin(top_10_cats)
].groupby('product_category_name_english').agg({
    'processing_time': 'mean',
    'seller_handling_time': 'mean',
    'carrier_shipping_time': 'mean'
}).reset_index()


# --- 4. Prepare data for plotting and visualize ---
# 'Melt' the DataFrame to make it suitable for Plotly Express
avg_times_melted = avg_times_by_cat.melt(
    id_vars='product_category_name_english',
    value_vars=['processing_time', 'seller_handling_time', 'carrier_shipping_time'],
    var_name='delivery_stage',
    value_name='average_days'
)

# Create the stacked bar chart
import plotly.express as px

fig = px.bar(
    avg_times_melted,
    x='product_category_name_english',
    y='average_days',
    color='delivery_stage',
    title='Average Delivery Stage Duration by Product Category (Top 10)',
    labels={
        'product_category_name_english': 'Product Category',
        'average_days': 'Average Duration (Days)',
        'delivery_stage': 'Delivery Stage'
    },
    category_orders={ # Sort the stages logically
        'delivery_stage': ['processing_time', 'seller_handling_time', 'carrier_shipping_time']
    }
)

fig.show()

In [ ]:
# To get the total count of unique cities
city_count = geolocation['geolocation_state'].nunique()
print(f"There are {city_count} unique city names in the geolocation table.")

# To list the first 15 unique city names as an example
print("\nSample of city names:")
print(geolocation['geolocation_city'].unique()[:15])

There are 73 unique city names in the geolocation table.

Sample of city names:
['sao paulo' 'são paulo' 'sao bernardo do campo' 'jundiaí'
 'taboão da serra' 'sãopaulo' 'sp' 'sa£o paulo' 'sao jose dos campos'
 'osasco' 'carapicuíba' 'carapicuiba' 'barueri' 'santana de parnaiba'
 'pirapora do bom jesus']


In [ ]:
import pandas as pd

# --- Configuration ---
REVIEWS_FILE_PATH = "E-commerce/order_reviews_dataset.csv"

complaint_keywords = {
    # Focuses only on specific terms for lateness.
    "Late Delivery": [
        "atraso", "atrasado", "demorou", "demora", "lento", "extraviado"
    ],
    # Uses very specific phrases for non-receipt.
    "Product Not Received": [
        "não recebi", "nao recebi", "não entregue", "nao entregue", "nunca chegou", "consta como entregue", "caixa vazia"
    ],
    # Focuses on words describing a broken or non-functional product.
    "Bad Product Quality / Defective": [
        "defeito", "quebrado", "quebrada", "nao funciona", "avariado", "estragado", "falso", "mal acabado", "baixa qualidade"
    ],
    # Uses phrases that clearly indicate the wrong item was sent.
    "Wrong Item Sent": [
        "veio errado", "item errado", "cor errada", "modelo errado", "veio outro", "produto trocado"
    ],
    # Focuses on words related to quantity and incomplete orders.
    "Missing Items / Partial Delivery": [
        "faltando", "incompleto", "só recebi um", "parcial", "faltou", "recebi a menos"
    ],
    # Uses keywords specific to the return/refund process.
    "Return & Refund Issues": [
        "devolver", "devolução", "troca", "dinheiro de volta", "cancelar", "reembolso", "estorno"
    ],
    # Focuses on complaints about the seller or lack of communication.
    "Poor Customer Service": [
        "sem resposta", "ninguem responde", "pessimo atendimento", "pós venda", "mau vendedor"
    ],
    # Focuses on the description/advertisement vs. the real product.
    "Different from Expectations": [
        "propaganda enganosa", "nao e como na foto", "descrição errada", "não corresponde"
    ]
}

# The categorization function
def categorize_complaint(comment):
    if not isinstance(comment, str):
        return "Other"
    
    comment_lower = comment.lower()
    scores = {category: 0 for category in complaint_keywords}
    
    for category, keywords in complaint_keywords.items():
        for keyword in keywords:
            if keyword in comment_lower:
                scores[category] += 1
    
    max_score = max(scores.values())
    
    if max_score == 0:
        return "Other"
    
    # Find all categories that have the max score
    best_categories = [category for category, score in scores.items() if score == max_score]
    
    # Return the first one in case of a tie
    return best_categories[0]

# --- Main Script ---
if __name__ == "__main__":
    print("--- Starting Category Debugging Process ---")
    
    print(f"\n[1/4] Loading reviews from '{REVIEWS_FILE_PATH}'...")
    reviews_df = pd.read_csv(REVIEWS_FILE_PATH)
    
    print("\n[2/4] Filtering for negative reviews with comments...")
    negative_reviews = reviews_df[reviews_df['review_score'] <= 3].dropna(subset=['review_comment_message']).copy()
    
    print("\n[3/4] Categorizing reviews...")
    negative_reviews['complaint_category'] = negative_reviews['review_comment_message'].apply(categorize_complaint)
    
    print("\n[4/4] Generating samples for each category...")
    
    # Get all categories that were assigned, including 'Other'
    all_categories = negative_reviews['complaint_category'].unique()
    
    print("\n--- SAMPLE REVIEWS PER CATEGORY ---")
    
    for category in sorted(all_categories):
        print("\n" + "="*20)
        print(f"CATEGORY: {category}")
        print("="*20)
        
        category_df = negative_reviews[negative_reviews['complaint_category'] == category]
        
        # Determine the sample size (up to 10)
        sample_size = min(10, len(category_df))
        
        # Get a random sample of comments
        sample_comments = category_df['review_comment_message'].sample(n=sample_size, random_state=42).tolist()
        
        # Print the numbered list of samples
        for i, comment in enumerate(sample_comments, 1):
            print(f"{i}. {comment}")
            
    print("\n--- Process Finished ---")

--- Starting Category Debugging Process ---

[1/4] Loading reviews from 'E-commerce/order_reviews_dataset.csv'...

[2/4] Filtering for negative reviews with comments...

[3/4] Categorizing reviews...

[4/4] Generating samples for each category...

--- SAMPLE REVIEWS PER CATEGORY ---

CATEGORY: Bad Product Quality / Defective
1. Produto c defeito, tentei por várias vezes, mas foi muitas, devolver o produto abrindo chamado na lannister e não consegui.
2. Recebi o produto, dentro do prazo, mas avalio de baixa qualidade. É necessário MUITO cuidado no manuseio, pois o material é muito frágil. No site não é possível fazer comparação da qualidade.
3. O PRODUTO CHEGOU ANTES DO PRAZO, POREM COM DEFEITO. LIGA, MÁS NÃO EXERCE SUA FUNÇÕES, ATRIBUO SUPOSTAMENTE O DEFEITO A MÁ QUALIDADE NO EMBALAMENTO, POIS CAIXA CHEGOU BASTANTE AMASSADA. 
4. Comprei um conjunto de pinças para alongamento de Cílios que me deixou extremamente insatisfeita! Logo que peguei nas mãos vi a péssima qualidade, cheio de def

In [6]:
import pandas as pd
import plotly.express as px
import numpy as np

# --- 1. Data Loading ---
def load_data():
    """Loads all necessary e-commerce datasets."""
    path = "E-commerce/"
    try:
        orders = pd.read_csv(path + "orders_dataset.csv")
        order_items = pd.read_csv(path + "order_items_dataset.csv")
        products = pd.read_csv(path + "products_dataset.csv")
        reviews = pd.read_csv(path + "order_reviews_dataset.csv")
        cat_trans = pd.read_csv(path + "product_category_name_translation.csv")
    except FileNotFoundError as e:
        print(f"Error loading data: {e}")
        print("Please make sure the script is in the correct folder and the 'E-commerce' directory exists.")
        return None
    return orders, order_items, products, reviews, cat_trans

# --- 2. Analysis Functions ---

def plot_freight_vs_score(df):
    """Analyzes and plots how freight value impacts review scores."""
    print("Analyzing: Freight Value vs. Review Score...")
    
    order_freight = df.groupby('order_id')['freight_value'].sum().reset_index()
    df_analysis = df[['order_id', 'review_score']].drop_duplicates().merge(order_freight, on='order_id')
    df_analysis.dropna(inplace=True)

    # <-- ADDED: Calculate correlation on the unbinned data -->
    correlation = df_analysis['freight_value'].corr(df_analysis['review_score'])

    bins = [0, 10, 20, 30, 40, 50, np.inf]
    labels = ["0-10", "10-20", "20-30", "30-40", "40-50", "50+"]
    df_analysis['freight_bin'] = pd.cut(df_analysis['freight_value'], bins=bins, labels=labels, right=False)
    
    binned_scores = df_analysis.groupby('freight_bin')['review_score'].mean().reset_index()
    
    fig = px.bar(
        binned_scores,
        x='freight_bin',
        y='review_score',
        # <-- ADDED: Correlation added to title -->
        title=f"Average Review Score vs. Shipping Cost (Correlation: {correlation:.2f})",
        labels={'freight_bin': 'Freight Value (R$)', 'review_score': 'Average Review Score'},
        text_auto='.2f'
    )
    fig.show()

def plot_photos_vs_score(df):
    """Analyzes and plots how the number of product photos impacts review scores."""
    print("Analyzing: Number of Photos vs. Review Score...")
    
    # <-- ADDED: Calculate correlation on the raw data -->
    correlation = df['product_photos_qty'].corr(df['review_score'])
    
    df_analysis = df.groupby('product_photos_qty')['review_score'].mean().reset_index()
    
    fig = px.bar(
        df_analysis,
        x='product_photos_qty',
        y='review_score',
        # <-- ADDED: Correlation added to title -->
        title=f"Average Review Score by Number of Photos (Correlation: {correlation:.2f})",
        labels={'product_photos_qty': 'Number of Photos', 'review_score': 'Average Review Score'},
        text_auto='.2f'
    )
    fig.show()

def plot_freight_vs_sales(df):
    """Analyzes and plots how freight value impacts sales volume and revenue."""
    print("Analyzing: Freight Value vs. Sales...")

    df_analysis = df.groupby('product_category_name_english').agg(
        avg_freight=('freight_value', 'mean'),
        total_revenue=('price', 'sum'),
        number_of_sales=('order_id', 'count')
    ).reset_index()

    # <-- ADDED: Calculate correlation on the aggregated category data -->
    correlation = df_analysis['avg_freight'].corr(df_analysis['number_of_sales'])

    fig = px.scatter(
        df_analysis,
        x='avg_freight',
        y='number_of_sales',
        size='total_revenue',
        hover_name='product_category_name_english',
        # <-- ADDED: Correlation added to title -->
        title=f"Sales Volume vs. Avg. Freight Value (Correlation: {correlation:.2f})",
        labels={'avg_freight': 'Average Freight Value (R$)', 'number_of_sales': 'Total Number of Items Sold'}
    )
    fig.show()

def plot_photos_vs_sales(df):
    """Analyzes and plots how the number of product photos impacts sales revenue."""
    print("Analyzing: Number of Photos vs. Sales Revenue...")

    # <-- ADDED: Calculate correlation on the raw data -->
    correlation = df['product_photos_qty'].corr(df['price'])

    df_analysis = df.groupby('product_photos_qty')['price'].sum().reset_index()
    df_analysis.rename(columns={'price': 'total_revenue'}, inplace=True)
    
    fig = px.bar(
        df_analysis,
        x='product_photos_qty',
        y='total_revenue',
        # <-- ADDED: Correlation added to title -->
        title=f"Total Revenue by Number of Photos (Correlation: {correlation:.2f})",
        labels={'product_photos_qty': 'Number of Photos', 'total_revenue': 'Total Revenue (R$)'}
    )
    fig.show()

# --- 3. Main Execution ---
if __name__ == "__main__":
    # Load the data
    data = load_data()
    if data:
        orders, order_items, products, reviews, cat_trans = data

        # Prepare a master DataFrame for analysis
        print("Preparing master data file for analysis...")
        items_products = order_items.merge(products, on='product_id', how='left')
        items_translated = items_products.merge(cat_trans, on='product_category_name', how='left')
        df_merged = items_translated.merge(reviews, on='order_id', how='left')
        df_merged.dropna(subset=['review_score', 'product_photos_qty', 'product_category_name_english'], inplace=True)
        print("Data preparation complete.")

        # Run and display all analyses
        plot_freight_vs_score(df_merged)
        plot_photos_vs_score(df_merged)
        plot_freight_vs_sales(df_merged)
        plot_photos_vs_sales(df_merged)

        print("\nAll analyses complete. Check your browser for the plots.")

Preparing master data file for analysis...
Data preparation complete.
Analyzing: Freight Value vs. Review Score...


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11404\647972712.py:38: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Analyzing: Number of Photos vs. Review Score...


Analyzing: Freight Value vs. Sales...


Analyzing: Number of Photos vs. Sales Revenue...



All analyses complete. Check your browser for the plots.


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# --- 1. Data Loading ---
def load_data():
    """Loads all necessary e-commerce datasets."""
    path = "E-commerce/"
    try:
        orders = pd.read_csv(path + "orders_dataset.csv")
        order_items = pd.read_csv(path + "order_items_dataset.csv")
        customers = pd.read_csv(path + "customers_dataset.csv")
        sellers = pd.read_csv(path + "sellers_dataset.csv")
        reviews = pd.read_csv(path + "order_reviews_dataset.csv")
        geolocation = pd.read_csv(path + "geolocation_dataset.csv")
    except FileNotFoundError as e:
        print(f"Error loading data: {e}")
        return None
    return orders, order_items, customers, sellers, reviews, geolocation

# --- 2. Distance Calculation Function ---
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculates the distance between two points on Earth in kilometers."""
    R = 6371  # Earth radius in kilometers
    
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c
    return distance

# --- 3. Main Execution ---
if __name__ == "__main__":
    # Load the data
    data = load_data()
    if data:
        orders, order_items, customers, sellers, reviews, geolocation = data

        # --- Data Preparation ---
        print("Preparing data... (This may take a moment)")
        
        # 1. Prepare geolocation data (get one coordinate pair per zip prefix)
        geo = geolocation.groupby('geolocation_zip_code_prefix').agg({
            'geolocation_lat': 'mean',
            'geolocation_lng': 'mean'
        }).reset_index()

        # 2. Get coordinates for customers and sellers
        customers_geo = customers.merge(geo, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix')
        sellers_geo = sellers.merge(geo, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix')

        # 3. Create a master order table with locations
        # For simplicity, we'll use the first seller for orders with multiple sellers
        order_sellers = order_items.drop_duplicates(subset='order_id', keep='first')[['order_id', 'seller_id']]
        
        df_merged = orders.merge(customers_geo[['customer_id', 'geolocation_lat', 'geolocation_lng']], on='customer_id')
        df_merged = df_merged.merge(order_sellers, on='order_id')
        df_merged = df_merged.merge(sellers_geo[['seller_id', 'geolocation_lat', 'geolocation_lng']], on='seller_id', suffixes=('_customer', '_seller'))

        # 4. Calculate the distance for each order
        df_merged['distance_km'] = haversine_distance(
            df_merged['geolocation_lat_customer'], df_merged['geolocation_lng_customer'],
            df_merged['geolocation_lat_seller'], df_merged['geolocation_lng_seller']
        )
        
        # 5. Link distance to review score
        df_analysis = df_merged.merge(reviews[['order_id', 'review_score']], on='order_id').dropna(subset=['distance_km', 'review_score'])
        print("Data preparation complete.")

        # --- Analysis and Plotting ---
        print("Analyzing: Order Distance vs. Review Score...")
        
        # Calculate overall correlation
        correlation = df_analysis['distance_km'].corr(df_analysis['review_score'])
        
        # Create bins for distance to visualize the trend
        bins = [0, 100, 300, 600, 1000, 2000, np.inf]
        labels = ["0-100", "100-300", "300-600", "600-1k", "1k-2k", "2k+"]
        df_analysis['distance_bin'] = pd.cut(df_analysis['distance_km'], bins=bins, labels=labels, right=False)
        
        # Calculate average score for each distance bin
        binned_scores = df_analysis.groupby('distance_bin')['review_score'].mean().reset_index()
        
        fig = px.bar(
            binned_scores,
            x='distance_bin',
            y='review_score',
            title=f"Average Review Score by Order Distance (Correlation: {correlation:.2f})",
            labels={'distance_bin': 'Distance from Seller to Customer (km)', 'review_score': 'Average Review Score'},
            text_auto='.2f'
        )
        fig.show()
        print("\nAnalysis complete. Check your browser for the plot.")